# Data Preparation

National Centers for Environmental Information (NCEI) is responsible for hosting and providing access to one of the most significant archives on Earth, with comprehensive oceanic, atmospheric, and geophysical data. From the depths of the ocean to the surface of the sun and from million-year-old ice core records to near real-time satellite images, NCEI is the Nation’s leading authority for environmental information.

The Global Summary of the Month (GSOM) and Global Summary of the Year (GSOY) datasets consist of 55 climatological variables computed from summary of the day observations of the Global Historical Climatology Network Daily dataset. Of these, 53 are monthly and annual summary variables and two are season-to-date variables. 

This project consumes GSOY data. In order to keep volume of data under control, we only consider precipitation and temperature data for US. 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Import necessary libraries

In [2]:
import os
import pandas as pd
from collections import OrderedDict
from sqlalchemy import create_engine, inspect
import tarfile

### Path to historic yearly weather information

In [3]:
RAW_DATA_FILE=os.path.join("..", "data", "gsoy-latest.tar.gz")

### Process archive downloaded from NCEI site - https://www.ncei.noaa.gov/data/gsoy/archive/

In [4]:
final_df = pd.DataFrame()

with tarfile.open(RAW_DATA_FILE, "r:gz") as tar:
    for index, member in enumerate(tar.getmembers()):
        # Extract only US data
        if not member.name.startswith('US'):
            continue
        csv = tar.extractfile(member)
        member_df = pd.read_csv(csv)
        # Skip records with no PRCP or TAVG. If record has TAVG, it also has TMAX and TMIN
        if 'TAVG' not in member_df.columns or 'PRCP' not in member_df.columns:
            continue
        
        member_df['NAME'] = member_df['NAME'].str.replace(' US', '')
        member_df['STATE'] = member_df['NAME'].apply(lambda x: x.split(',')[-1].strip())
        member_df['NAME'] = member_df['NAME'].apply(lambda x: x.split(',')[0].strip())
        
        # Use 1923 and later for analysis
        member_df = member_df[member_df.DATE >= 1923]
        
        # Extract only columns that are relevant for this analysis
        COLS_OF_INTEREST = ['DATE', 'NAME', 'STATE', 'STATION', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'PRCP', 'TAVG', 'TMAX', 'TMIN']
        final_df = pd.concat([final_df, member_df[COLS_OF_INTEREST]], sort=False)
        #if index > 1:
        #    break
final_df.head()

,DATE,NAME,STATE,STATION,LATITUDE,LONGITUDE,ELEVATION,PRCP,TAVG,TMAX,TMIN
0,2009,SIOUX FALLS ENVIRON. CANADA,SD,US009052008,43.7333,-96.6333,482.0,549.7,6.26,12.07,0.45
1,2010,SIOUX FALLS ENVIRON. CANADA,SD,US009052008,43.7333,-96.6333,482.0,755.1,7.32,12.91,1.72
2,2011,SIOUX FALLS ENVIRON. CANADA,SD,US009052008,43.7333,-96.6333,482.0,632.0,7.12,13.00,1.25
3,2012,SIOUX FALLS ENVIRON. CANADA,SD,US009052008,43.7333,-96.6333,482.0,NaN,9.75,16.33,3.17
4,2013,SIOUX FALLS ENVIRON. CANADA,SD,US009052008,43.7333,-96.6333,482.0,547.5,6.34,12.06,0.62


In [5]:
final_df.shape

(517848, 11)

In [6]:
final_df = final_df.sort_values(by=['DATE'])

In [7]:
final_df.describe()

,DATE,LATITUDE,LONGITUDE,ELEVATION,PRCP,TAVG,TMAX,TMIN
count,517848.000000,517848.000000,517848.000000,517199.000000,466716.000000,366068.000000,385394.000000,381673.000000
mean,1975.390423,39.520148,-98.727385,619.656175,860.217386,11.293766,17.835929,4.770710
std,26.024874,6.268827,17.686655,712.982492,504.208733,5.275770,5.488174,5.417967
min,1923.000000,19.061400,-177.350000,-59.100000,0.000000,-15.450000,-12.490000,-19.910000
25%,1955.000000,35.414000,-110.866670,146.300000,466.600000,7.610000,14.060000,0.950000
50%,1976.000000,39.667900,-96.768600,316.400000,834.000000,10.780000,17.360000,4.470000
75%,1998.000000,43.377200,-85.412100,887.000000,1146.400000,15.020000,21.870000,8.330000
max,2018.000000,71.321300,179.283330,13109.400000,9628.500000,26.970000,34.660000,24.270000


### Handle Missing Values 
Use forward propagation to use previous value in the series to fill missing value

In [8]:
# forward-fill
final_df = final_df.fillna(method='ffill')

### Write to SQLite DB

In [9]:
final_df = final_df.reset_index().drop(labels='index', axis=1) # Remove duplicate indices
engine = create_engine('sqlite:///../db/adventurously.sqlite', echo=False)
final_df.to_sql('CLIMATE_HISTORY', if_exists='append', con=engine, index=True)

In [10]:
engine = create_engine('sqlite:///../db/adventurously.sqlite', echo=False)
pd.read_sql_query('select * from CLIMATE_HISTORY', con=engine)

,index,STATION,DATE,NAME,STATE,LATITUDE,LONGITUDE,ELEVATION,PRCP,TAVG,TMAX,TMIN
0,0,USC00307317,1923,ROXBURY,NY,42.283330,-74.566670,454.2,1043.6,NaN,NaN,NaN
1,1,USC00304024,1923,HUDSON,NY,42.250000,-73.783330,98.1,1043.6,NaN,NaN,2.16
2,2,USC00404609,1923,JEFFERSON CITY EVAP,TN,36.116670,-83.500000,366.1,1040.0,NaN,NaN,2.16
3,3,USC00250760,1923,BENKELMAN,NE,40.049970,-101.532580,909.5,682.1,NaN,NaN,2.16
4,4,USC00175875,1923,NORTH BRIDGTON,ME,44.133330,-70.716670,136.9,1027.0,6.54,12.96,0.12
5,5,USC00452675,1923,EVERETT,WA,47.975200,-122.195000,18.3,730.2,9.95,14.22,5.69
6,6,USC00029420,1923,WINKELMAN 6 S,AZ,32.916670,-110.716670,634.0,730.2,17.83,26.30,9.35
7,7,USC00345509,1923,MANGUM,OK,34.891000,-99.501600,486.2,1146.4,16.70,24.31,9.08
8,8,USC00233940,1923,HOLLISTER,MO,36.616667,-93.233333,486.2,1085.2,16.70,24.31,9.08
9,9,USC00054945,1923,LEROY 5 WSW,CO,40.514610,-102.990680,1361.5,674.1,8.64,15.77,1.50


In [11]:
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
print(list(Base.classes))

[<class 'sqlalchemy.ext.automap.CLIMATE_HISTORY'>]
